In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
import xgboost as xgb
import lightgbm as lgb

from sklearn.metrics import classification_report, accuracy_score 
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, matthews_corrcoef
from sklearn.metrics import confusion_matrix

In [3]:
#importing total X_train(features)
X = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/X.csv')

In [4]:
#importing total Y_train(target)
y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/y.csv')

In [5]:
#model building and fitting
lgbm_clf = lgb.LGBMClassifier()
lgbm_clf.fit(X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMClassifier()

In [6]:
#train accuracy
lgbm_clf.score(X, y)

0.893325

In [7]:
yPred = lgbm_clf.predict(X)

In [8]:
#various performance scores
acc = accuracy_score(y, yPred)
print("The accuracy is {}".format(acc))
  
prec = precision_score(y, yPred)
print("The precision is {}".format(prec))
  
rec = recall_score(y, yPred)
print("The recall is {}".format(rec))
  
f1 = f1_score(y, yPred)
print("The F1-Score is {}".format(f1))
  
MCC = matthews_corrcoef(y, yPred)
print("The Matthews correlation coefficient is{}".format(MCC))

The accuracy is 0.893325
The precision is 0.6244350942808166
The recall is 0.100175
The F1-Score is 0.17265226102505546
The Matthews correlation coefficient is0.22004393512054898


In [9]:
#ks score function
def ks_(data=None,target=None, prob=None):
    data['target0'] = 1 - data[target]
    data['bucket'] = pd.qcut(data[prob], 10)
    grouped = data.groupby('bucket', as_index = False)
    kstable = pd.DataFrame()
    kstable['min_prob'] = grouped.min()[prob]
    kstable['max_prob'] = grouped.max()[prob]
    kstable['events']   = grouped.sum()[target]
    kstable['nonevents'] = grouped.sum()['target0']
    kstable = kstable.sort_values(by="min_prob", ascending=False).reset_index(drop = True)
    kstable['event_rate'] = (kstable.events / data[target].sum()).apply('{0:.2%}'.format)
    kstable['nonevent_rate'] = (kstable.nonevents / data['target0'].sum()).apply('{0:.2%}'.format)
    kstable['cum_eventrate']=(kstable.events / data[target].sum()).cumsum()
    kstable['cum_noneventrate']=(kstable.nonevents / data['target0'].sum()).cumsum()
    kstable['KS'] = np.round(kstable['cum_eventrate']-kstable['cum_noneventrate'], 3) * 100
    #Formating
    kstable['cum_eventrate']= kstable['cum_eventrate'].apply('{0:.2%}'.format)
    kstable['cum_noneventrate']= kstable['cum_noneventrate'].apply('{0:.2%}'.format)
    kstable.index = range(1,11)
    kstable.index.rename('Decile', inplace=True)
    pd.set_option('display.max_columns', 9)
    print(kstable)
    
    return 

In [10]:
#creating data frame to check ks score for total train data
ks=pd.DataFrame(lgbm_clf.predict_proba(X))
del ks[0]

In [11]:
ks['target']=y

#ks table
ks_(ks,'target',1)

        min_prob  max_prob   events  nonevents event_rate nonevent_rate  \
Decile                                                                    
1       0.234942  0.863598  28755.0    43245.0     35.94%         6.76%   
2       0.148861  0.234937  14090.0    57910.0     17.61%         9.05%   
3       0.109280  0.148861   9516.0    62484.0     11.89%         9.76%   
4       0.086887  0.109278   7029.0    64971.0      8.79%        10.15%   
5       0.072061  0.086887   5596.0    66404.0      7.00%        10.38%   
6       0.061277  0.072060   4379.0    67621.0      5.47%        10.57%   
7       0.052539  0.061277   3708.0    68292.0      4.63%        10.67%   
8       0.044599  0.052539   2979.0    69021.0      3.72%        10.78%   
9       0.036252  0.044599   2394.0    69606.0      2.99%        10.88%   
10      0.007642  0.036252   1554.0    70446.0      1.94%        11.01%   

       cum_eventrate cum_noneventrate    KS  
Decile                                       
1      

In [12]:
#creating data frame of cust ids with probabilities
X_t= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/X_t.csv')
test_cust= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_cust.csv')
result_model = pd.DataFrame(lgbm_clf.predict_proba(X_t))
result=pd.concat([test_cust['cust_id'],result_model[1] ], axis=1)
result.rename(columns={1:'target'},inplace=True)

In [13]:
result

,cust_id,target
0,10146566,0.116837
1,10203613,0.054079
2,10322129,0.021052
3,10373212,0.063938
4,10487539,0.052480
...,...,...
179995,45739460,0.038530
179996,56006782,0.028555
179997,61138340,0.035148
179998,65715392,0.027892


In [ ]:
#uploading results(probabilities)
result.to_csv('/content/drive/MyDrive/Colab Notebooks/results.csv',index=False)